In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import ast
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

In [2]:
# load dataset
train = pd.read_csv('../20043374/trainProcessed.csv')
validate = pd.read_csv('../20043374/validateProcessed.csv')
test = pd.read_csv('../20043374/testProcessed.csv')

In [3]:
def convert_list_diagnosis(diagnosis_list):
    if isinstance(diagnosis_list, str):
        try:
            return ast.literal_eval(diagnosis_list)
        except Exception as e: 
            return None 
    else:
        return None 

In [4]:
train['DIFFERENTIAL_DIAGNOSIS'] = train['DIFFERENTIAL_DIAGNOSIS'].apply(convert_list_diagnosis)
validate['DIFFERENTIAL_DIAGNOSIS'] = validate['DIFFERENTIAL_DIAGNOSIS'].apply(convert_list_diagnosis)
test['DIFFERENTIAL_DIAGNOSIS'] = test['DIFFERENTIAL_DIAGNOSIS'].apply(convert_list_diagnosis)

In [5]:
def one_hot_encode_diagnosis(df, column_name):
    # ensure that each diagnosis list is correctly formatted
    df[column_name] = df[column_name].apply(lambda x: x if isinstance(x, list) else [])

    # flatten the list of all possible diagnoses, taking only the diagnosis name (first item of each sublist)
    all_diagnoses = set(diagnosis[0] for sublist in df[column_name] for diagnosis in sublist if isinstance(diagnosis, list) and len(diagnosis) > 0)
    
    # initialize a dictionary to hold the one-hot encoded data
    one_hot_encoded_data = {diagnosis: [] for diagnosis in all_diagnoses}
    
    # populate the dictionary with 1s and 0s based on diagnosis presence
    for index, row in df.iterrows():
        present_diagnoses = {diagnosis[0] for diagnosis in row[column_name] if isinstance(diagnosis, list) and len(diagnosis) > 0}
        for diagnosis in all_diagnoses:
            one_hot_encoded_data[diagnosis].append(1 if diagnosis in present_diagnoses else 0)
    
    # convert the dictionary to a DataFrame
    one_hot_y = pd.DataFrame(one_hot_encoded_data)
    
    return one_hot_y

In [6]:
train_y = one_hot_encode_diagnosis(train, 'DIFFERENTIAL_DIAGNOSIS')
validate_y = one_hot_encode_diagnosis(validate, 'DIFFERENTIAL_DIAGNOSIS')
test_y = one_hot_encode_diagnosis(test, 'DIFFERENTIAL_DIAGNOSIS')

In [7]:
def parse_age(x):
    if x <= 4:
        return 1
    elif x <= 15:
        return 2
    elif x <= 30:
        return 3
    elif x <= 45:
        return 4
    elif x <= 60:
        return 5
    else:
        return 6

In [8]:
train = train.drop(['PATHOLOGY', 'DIFFERENTIAL_DIAGNOSIS', 'INITIAL_EVIDENCE'], axis=1)
validate = validate.drop(['PATHOLOGY', 'DIFFERENTIAL_DIAGNOSIS', 'INITIAL_EVIDENCE'], axis=1)
test = test.drop(['PATHOLOGY', 'DIFFERENTIAL_DIAGNOSIS', 'INITIAL_EVIDENCE'], axis=1)

In [9]:
train['AGE'] = train['AGE'].apply(parse_age)
validate['AGE'] = validate['AGE'].apply(parse_age)
test['AGE'] = test['AGE'].apply(parse_age)

In [10]:
train = train.replace(0, -1)
validate = validate.replace(0, -1)
test = test.replace(0, -1)

In [11]:
import pickle
train.astype(np.int8).to_pickle('train.zst')
validate.astype(np.int8).to_pickle('validate.zst')
test.astype(np.int8).to_pickle('test.zst')

In [11]:
# train['SEX'] = train['SEX'].replace(-1, 0)
# validate['SEX'] = validate['SEX'].replace(-1, 0)
# test['SEX'] = test['SEX'].replace(-1, 0)

In [2]:
import zstandard as zstd
import pickle
with open('train.zst', 'rb') as compressed_file:
    decompressor = zstd.ZstdDecompressor()
    with decompressor.stream_reader(compressed_file) as reader:
        decompressed_data = reader.read()

train_wo_sex = pickle.loads(decompressed_data).drop('SEX', axis=1)
train_wo_sex = train_wo_sex.sort_index(axis=1)

In [3]:
train_wo_sex = train_wo_sex.replace(-1, 0)

In [3]:
prefix_to_columns = {}
for n, col in enumerate(train_wo_sex.columns):
    if '_@_' in col:
        prefix, _ = col.split('_@_', 1)
        if prefix not in prefix_to_columns:
            prefix_to_columns[prefix] = []
        prefix_to_columns[prefix].append(n)
        
column_to_prefix = {}
for n, col in enumerate(train_wo_sex.columns):
    if '_@_' in col:
        prefix, _ = col.split('_@_', 1)
        column_to_prefix[n] = prefix
    else:
        column_to_prefix[n] = None
        
prefix_group_counts = {}
for col in train_wo_sex.columns:
    if '_@_' in col:
        prefix, _ = col.split('_@_', 1)
        prefix_group_counts[prefix] = prefix_group_counts.get(prefix, 0) + 1

In [5]:
from scipy.stats import entropy
import torch.nn.functional as F

def calculate_entropy(probabilities):
    # Ensure no zero probabilities; add a small value
    probabilities = probabilities.clamp(min=1e-9)
    return -(probabilities * probabilities.log2()).sum(dim=1)

# Function to return the n columns with the highest entropies.
def calculate_top_entropy_columns(df, columns, top_n=1):
    
    # df_tensor = torch.where(df == -1, torch.tensor(0, device=df.device), df)
    prob = df.float().mean(dim=0)
    prob = torch.stack([1 - prob, prob], dim=1)
    entropy_values = calculate_entropy(prob)
    index_mapping = {value: index for index, value in enumerate(columns)}
    entropy_values_group = torch.zeros(len(columns), device=df.device)
    
    maximun = 0
    max_col = None
    processed_groups = set()
    for n, col in enumerate(columns):
        if column_to_prefix[col]:
            group_prefix = column_to_prefix.get(col)
            # Skip if the group has already been calculated
            if group_prefix in processed_groups:
                continue
            indexes = [index_mapping[i] for i in prefix_to_columns[group_prefix]]
            entropy_value = entropy_values[indexes].sum()
            entropy_values_group[indexes] = entropy_value
            processed_groups.add(group_prefix)
        else:
            entropy_value = entropy_values[n]
            entropy_values_group[n] = entropy_value
        # if entropy_value >= maximun:
        #     maximun = entropy_value
        #     max_col = col
    max_entropy_group_id = torch.argmax(entropy_values_group)
    max_col = columns[max_entropy_group_id]
    maximun = entropy_values_group[max_entropy_group_id].item()
        
    return (max_col, maximun)

In [49]:
aa, en = calculate_top_entropy_columns(train_wo_sex)
print(len(aa), en)
b = train_wo_sex.drop(aa, axis=1)
cc, en = calculate_top_entropy_columns(b)
d = b.drop(cc, axis=1)
print(len(cc), en)
ee, en = calculate_top_entropy_columns(d)
f = d.drop(ee, axis=1)
print(len(ee), en)
gg, en = calculate_top_entropy_columns(f)
print(len(gg), en)
# len(calculate_top_entropy_columns(bb.drop(cc, axis=1)))

16 5.451161490500922
104 4.18163624932129
11 3.352121841444533
11 3.346291305087514


In [6]:
import torch

train_wo_sex_tensor = torch.tensor(train_wo_sex.values, dtype=torch.int32).cuda()
n = len(train_wo_sex)
age_idx = train_wo_sex.columns.get_loc('AGE')  # This is for pandas, done once before conversion
original_column_names = train_wo_sex.columns.copy()
train_wo_sex.columns = range(len(train_wo_sex.columns))
train_wo_age = train_wo_sex.copy()
train_wo_age[age_idx] = 0

In [17]:
from sklearn.metrics.pairwise import cosine_similarity, pairwise_distances
from scipy.sparse import csr_matrix
from tqdm import tqdm
import os
from DPTreeNode import DPTreeNode

# len(train_wo_sex)
train_wo_age_tensor = train_wo_sex_tensor.clone()
train_wo_age_tensor[:, age_idx] = 0
# col_set = set(train_wo_sex.columns)

if os.path.isfile('inquiry_system.pkl'):
    tree_root = pickle.load(open('inquiry_system.pkl', 'rb'))
    print('Tree loaded!')
else:
    reveal, en = calculate_top_entropy_columns(train_wo_age_tensor, list(train_wo_sex.columns))
    tree_root = DPTreeNode(None, reveal)
    tree_root.entropy = en

result = None
small_chunk = None
large_chunk = None
stored_tree_param = 10000
num_question = 20

# len(train_wo_sex)
for idx in tqdm(range(0, 2)):
    instance_tensor = train_wo_sex_tensor[idx]
    instance = train_wo_sex.iloc[idx]
    percent = 500
    root = tree_root
    reveal = []
    # dataset = train_wo_age
    similarity = None
    instance_wo_age = train_wo_age_tensor[idx]
    if idx == stored_tree_param:
        pickle.dump(root, open('inquiry_system.pkl', 'wb'))
        stored_tree_param += 10000
    for i in range(num_question):
        
        new_inquiry = root.next_question
        if column_to_prefix[new_inquiry]:
            new_inquiry = prefix_to_columns[column_to_prefix[new_inquiry]]
        else:
            new_inquiry = [new_inquiry]
            
        reveal += new_inquiry

        if i == 3:
            # answer = "".join(str(x) for x in instance_tensor[new_inquiry + [age_idx]].tolist())
            answer = "".join(str(x) for x in instance[new_inquiry + [age_idx]].tolist())
            reveal += [age_idx]
            # dataset = train_wo_sex
        else:
            # answer = "".join(str(x) for x in instance_tensor[new_inquiry].tolist())
            answer = "".join(str(x) for x in instance[new_inquiry].tolist())
        next_node = root.look_up_children(answer)
        if next_node:
            root = next_node
        else: 
            if i == num_question - 1: continue
            cur_node = DPTreeNode(answer, None)
            root.add_child(cur_node)
            if similarity is not None:
                if i == 3:
                    for question_idx in new_inquiry:
                        similarity += 2 * (train_wo_age_tensor[:, question_idx] != instance_wo_age[question_idx])
                    similarity += (train_wo_sex_tensor[:, age_idx] - instance_tensor[age_idx]).abs()
                else:
                    for question_idx in new_inquiry:
                        similarity += 2 * (train_wo_age_tensor[:, question_idx] != instance_tensor[question_idx])
            else:
                similarity = torch.zeros(train_wo_sex_tensor.size(0), device="cuda")
                for question_idx in reveal:
                    if question_idx == age_idx:
                        similarity += (train_wo_sex_tensor[:, age_idx] - instance_tensor[age_idx]).abs()
                    else:
                        similarity += 2 * (train_wo_age_tensor[:, question_idx] != instance_wo_age[question_idx])
            k = n//10000*percent
            values, top_indices = torch.topk(similarity, k, largest=False)
            top_actual_indices = top_indices
            similar_cases = train_wo_age_tensor[top_actual_indices]
            selected_col = [col for col in train_wo_sex.columns if col not in reveal]
            new_inquiry, en = calculate_top_entropy_columns(similar_cases[:, selected_col], selected_col)
            cur_node.next_question = new_inquiry
            cur_node.entropy = np.int8(en//2)
            root = cur_node

        percent = percent//(1.4**(root.entropy + 1))
        percent = int(percent) if percent > 1 else 1
        torch.cuda.synchronize()
    # result = pd.concat([result, mask_instance.to_frame().T], axis=0)
    mask_instance = pd.Series(0, index=range(len(original_column_names)))
    mask_instance[reveal] = train_wo_sex.iloc[idx][reveal]
    if small_chunk is None:
        small_chunk = np.array([mask_instance.to_numpy()])
    else:
        small_chunk = np.vstack((small_chunk, mask_instance.to_numpy()))
    if idx % 100 == 99:
        if large_chunk is None:
            large_chunk = small_chunk
        else:
            large_chunk = np.vstack((large_chunk, small_chunk))
        small_chunk = None
        if idx % 10000 == 9999:
            if result is None:
                result = large_chunk
            else:
                result = np.vstack((result, large_chunk))
            large_chunk = None
    torch.cuda.synchronize()
            
# Empty the chunk
if large_chunk is not None:
    if result is None: 
        result = large_chunk
    else:
        result = np.vstack((result, large_chunk))
if small_chunk is not None:
    if result is None: 
        result = small_chunk
    else:
        result = np.vstack((result, small_chunk))
        
result = pd.DataFrame(result, columns=original_column_names)
result_file = 'result.zst'
if os.path.isfile(result_file):
    result_file = 'result_2.zst'
result.astype(np.int8).to_pickle(result_file)
pickle.dump(tree_root, open('inquiry_system.pkl', 'wb'))

Tree loaded!


100%|██████████| 2/2 [00:00<00:00, 119.80it/s]


In [18]:
instance[new_inquiry]

1   -1
Name: 1, dtype: int8

In [16]:
type(new_inquiry[0])

int

In [10]:
instance_tensor[new_inquiry].tolist()

[-1]

In [12]:
answer

'-1'

In [36]:
similar_cases.shape

torch.Size([102, 517])

In [50]:
import pandas as pd
import numpy as np
import torch
import time

# Sample size for the benchmark
n = 1000000  # Adjust this based on your typical dataset size

# Generating a sample DataFrame and mask instance
np.random.seed(0)
train_wo_sex = pd.DataFrame({
    'AGE': np.random.randint(18, 65, size=n),
    'QUESTION1': np.random.randint(0, 2, size=n),
    'QUESTION2': np.random.randint(0, 2, size=n),
})
mask_instance = pd.Series({'AGE': 30, 'QUESTION1': 1, 'QUESTION2': 0})
reveal = ['AGE', 'QUESTION1', 'QUESTION2']

# CPU Version
start_cpu = time.time()
similarity = pd.Series(0, index=train_wo_sex.index)
for question in reveal:
    if question == 'AGE':
        similarity += abs(train_wo_sex['AGE'] - mask_instance.loc[question])
    else:
        similarity += 2 * (train_wo_sex[question] != mask_instance.loc[question])
end_cpu = time.time()
time_cpu = end_cpu - start_cpu

# GPU Version
# start_gpu = time.time()
# Initial setup for GPU version as per the earlier provided code
similarity_gpu = torch.zeros(len(train_wo_sex), device='cuda')
train_columns_gpu = {col: torch.tensor(train_wo_sex[col].values, device='cuda') for col in reveal}
start_gpu = time.time()
mask_values_gpu = {question: torch.tensor(mask_instance.loc[question], device='cuda') for question in reveal}
for question in reveal:
    if question == 'AGE':
        similarity_gpu += torch.abs(train_columns_gpu[question] - mask_values_gpu[question])
    else:
        similarity_gpu += 2 * (train_columns_gpu[question] != mask_values_gpu[question]).float()
end_gpu = time.time()
time_gpu = end_gpu - start_gpu

# Print the times
print(f"CPU Time: {time_cpu} seconds")
print(f"GPU Time: {time_gpu} seconds")


CPU Time: 0.018001079559326172 seconds
GPU Time: 0.0010044574737548828 seconds


In [25]:
pickle.dump(tree_root, open('inquiry_system.pkl', 'wb'))

In [23]:
cur_node.parent.children = []

In [24]:
cur_node.parent.children

[]

In [7]:
len(tree_root.children)

78

In [55]:
len(result.iloc[0][result.iloc[0] != 0])

303

In [45]:
result.iloc[10][result.iloc[10] != 0].index

Index(['AGE', 'crach_sg', 'douleurxx', 'douleurxx_carac_@_NA',
       'douleurxx_carac_@_déchirante',
       'douleurxx_carac_@_lancinante_/_choc_électrique',
       'douleurxx_carac_@_pénible', 'douleurxx_carac_@_sensible',
       'douleurxx_carac_@_un_coup_de_couteau',
       'douleurxx_carac_@_un_tiraillement',
       ...
       'trav1_@_AfriqO', 'trav1_@_AfriqSS', 'trav1_@_AmerC', 'trav1_@_AmerN',
       'trav1_@_AmerS', 'trav1_@_Asie', 'trav1_@_AsieSSE', 'trav1_@_Cara',
       'trav1_@_Euro', 'trav1_@_N'],
      dtype='object', length=319)

In [ ]:
# 調整percent

In [13]:
similarity

0          54
1          62
2          70
3          62
4          64
           ..
1025597    70
1025598    70
1025599    70
1025600    70
1025601    70
Length: 1025602, dtype: int64

In [27]:
prefix_to_columns.keys()

dict_keys(['lesions_peau_endroitducorps', 'douleurxx_irrad', 'douleurxx_endroitducorps', 'douleurxx_precis', 'trav1', 'oedeme_endroitducorps', 'lesions_peau_plusqu1cm', 'douleurxx_carac', 'lesions_peau_prurit', 'lesions_peau_intens', 'douleurxx_soudain', 'lesions_peau_couleur', 'lesions_peau_elevee', 'douleurxx_intens', 'lesions_peau_desquame'])

In [28]:
root = tree_root
pickle.dump(root, open('inquiry_system.pkl', 'wb'))

### For Inference the Inquiry System

In [4]:
import zstandard as zstd
import pickle
with open('test.zst', 'rb') as compressed_file:
    decompressor = zstd.ZstdDecompressor()
    with decompressor.stream_reader(compressed_file) as reader:
        decompressed_data = reader.read()

test_wo_sex = pickle.loads(decompressed_data).drop('SEX', axis=1)

In [8]:
test_wo_sex.columns.

Index(['AGE', 'I30', 'diarrhee', 'bode',
       'lesions_peau_endroitducorps_@_face_dorsale_main_D_',
       'douleurxx_irrad_@_sous_la_machoire',
       'douleurxx_irrad_@_cartilage_thyroidien',
       'douleurxx_irrad_@_arrière_de_tête',
       'douleurxx_endroitducorps_@_hypochondre_G_',
       'douleurxx_endroitducorps_@_oreille_G_',
       ...
       'etourdissement', 'hernie_hiatale', 'douleurxx_irrad_@_trachée',
       'douleurxx_endroitducorps_@_orteil__1__G_', 'ww_dd',
       'lesions_peau_endroitducorps_@_petite_lèvre_G_',
       'lesions_peau_elevee_@_2', 'j17_j18', 'lesions_peau_intens_@_0',
       'lesions_peau_endroitducorps_@_vagin'],
      dtype='object', length=517)

In [5]:
from sklearn.metrics.pairwise import cosine_similarity, pairwise_distances
from scipy.sparse import csr_matrix
from tqdm import tqdm
import os
from DPTreeNode import DPTreeNode

n = len(train_wo_sex)
train_wo_age = train_wo_sex.copy()
train_wo_age['AGE'] = 0

if os.path.isfile('inquiry_system.pkl'):
    tree_root = pickle.load(open('inquiry_system.pkl', 'rb'))
    print('Tree loaded!')
else:
    reveal = calculate_top_entropy_columns(train_wo_age)
    tree_root = DPTreeNode(None, reveal)
result = None
small_chunk = None
large_chunk = None
num_question = 25


for idx, instance in tqdm(test_wo_sex.iterrows(), total=len(test_wo_sex)):
    percent = 1000
    root = tree_root
    reveal = []
    dataset = train_wo_age
    similarity = None
    for i in range(num_question):
        new_inquiry = root.next_question
        reveal += new_inquiry 

        if i == 7:
            answer = list("".join('0' if x==-1 else str(x) for x in instance[new_inquiry + ['AGE']]))
            reveal += ['AGE']
            dataset = train_wo_sex
        else:
            answer = instance[new_inquiry[0]]
            
        next_node = root.look_up_children(answer)
        if next_node:
            root = next_node
        else: 
            mask_instance = pd.Series(0, index=instance.index)
            mask_instance[reveal] = instance[reveal]
            if i == num_question - 1: continue
            cur_node = DPTreeNode(answer, None)
            root.add_child(cur_node)
            if similarity is not None:
                if i == 7:
                    f_answer = 1 if int(answer[0]) == 1 else -1
                    similarity += (train_wo_sex[new_inquiry[0]] != f_answer)
                    similarity += abs(train_wo_sex['AGE'] - int(answer[1]))
                else:
                    similarity += 2*(train_wo_sex[new_inquiry[0]] != answer)
            else:
                similarity = pd.Series(0, index=train_wo_sex.index)
                for question in reveal:
                    if question == 'AGE':
                        similarity += abs(train_wo_sex['AGE'] - mask_instance.loc[question])
                    similarity += 2*(train_wo_sex[question] != mask_instance.loc[question])

            cur_threshold = cur_node.parent.threshold + 2 if i != 7 else cur_node.parent.threshold + 7
            sorted_similarity = similarity[similarity <= cur_threshold].argsort() 
            similar_ind = sorted_similarity.iloc[:n//1000*percent]
            cur_node.threshold = similarity[similar_ind.iloc[-1]]
            
            similar_cases = dataset.iloc[similar_ind]
            new_inquiry = calculate_top_entropy_columns(similar_cases.drop(reveal, axis=1))
            cur_node.next_question = new_inquiry
            root = cur_node
        
        percent = percent//1.4
        percent = int(percent) if percent > 5 else 5
    # result = pd.concat([result, mask_instance.to_frame().T], axis=0)
    if small_chunk is None:
        small_chunk = np.array([mask_instance.to_numpy()])
    else:
        small_chunk = np.vstack((small_chunk, mask_instance.to_numpy()))
    if idx % 100 == 99:
        if large_chunk is None:
            large_chunk = small_chunk
        else:
            large_chunk = np.vstack((large_chunk, small_chunk))
        small_chunk = None
        if idx % 10000 == 9999:
            if result is None:
                result = large_chunk
            else:
                result = np.vstack((result, large_chunk))
            large_chunk = None
            
# Empty the chunk
if large_chunk is not None:
    if result is None: 
        result = large_chunk
    else:
        result = np.vstack((result, large_chunk))
if small_chunk is not None:
    if result is None: 
        result = small_chunk
    else:
        result = np.vstack((result, small_chunk))
        
result = pd.DataFrame(result, columns=train_wo_sex.columns)

Tree loaded!


100%|██████████| 134529/134529 [09:52<00:00, 227.08it/s]


In [7]:
result.astype(np.int8).to_pickle('test_result.zst')

In [6]:
result

,AGE,I30,diarrhee,bode,lesions_peau_endroitducorps_@_face_dorsale_main_D_,douleurxx_irrad_@_sous_la_machoire,douleurxx_irrad_@_cartilage_thyroidien,douleurxx_irrad_@_arrière_de_tête,douleurxx_endroitducorps_@_hypochondre_G_,douleurxx_endroitducorps_@_oreille_G_,...,etourdissement,hernie_hiatale,douleurxx_irrad_@_trachée,douleurxx_endroitducorps_@_orteil__1__G_,ww_dd,lesions_peau_endroitducorps_@_petite_lèvre_G_,lesions_peau_elevee_@_2,j17_j18,lesions_peau_intens_@_0,lesions_peau_endroitducorps_@_vagin
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134524,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
134525,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
134526,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
134527,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
tree_root.children[0].children[1].children[0].children[0].answer

-1

In [5]:
if os.path.isfile('inquiry_system.pkl'):
    tree_root = pickle.load(open('inquiry_system.pkl', 'rb'))
    print('Tree loaded!')

Tree loaded!


In [ ]:
store_tree = pickle.load(open('inquiry_system.pkl', 'rb'))

In [22]:
store_tree.children[2].answer

['3', '0', '1', '0', '1']